<a href="https://colab.research.google.com/github/malinphy/IMDB_Analysis_different_approachs/blob/main/IMDB_ML_VS_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import os
import re 

In [2]:
data_dir ='https://raw.githubusercontent.com/malinphy/IMDB_Analysis_different_approachs/main/IMDB_Dataset_short2.csv'

data = pd.read_csv(data_dir)

data['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [3]:
data_reviews = data['review'].copy()

In [4]:
from sklearn.model_selection import train_test_split

train_reviews , test_review = train_test_split(data_reviews , train_size= 0.8 , random_state= 42)

In [5]:
train_reviews.shape, test_review.shape

((15199,), (3800,))

In [6]:
for i in range(len(data_reviews)):
    data_reviews[i] = re.sub("<br /><br />|,|\?|\(|\)|\"|!|:|@|#|&|\+"," ",data_reviews[i])
    data_reviews[i] = re.sub("-|_|\."," ",data_reviews[i])
    data_reviews[i] = data_reviews[i].lower()
    data_reviews[i] = re.sub("1|2|3|4|5|6|7|8|9|0","",data_reviews[i])
    data_reviews[i] = re.sub("  "," ",data_reviews[i])
    data_reviews[i] = re.sub("   "," ",data_reviews[i])

In [7]:
data_reviews[1]

"a wonderful little production  the filming technique is very unassuming very old time bbc fashion and gives a comforting and sometimes discomforting sense of realism to the entire piece  the actors are extremely well chosen michael sheen not only has got all the polari but he has all the voices down pat too you can truly see the seamless editing guided by the references to williams' diary entries not only is it well worth the watching but it is a terrificly written and performed piece a masterful production about one of the great master's of comedy and his life  the realism really comes home with the little things the fantasy of the guard which rather than use the traditional 'dream' techniques remains solid then disappears it plays on our knowledge and our senses particularly with the scenes concerning orton and halliwell and the sets particularly of their flat with halliwell's murals decorating every surface are terribly well done "

In [8]:
from sklearn.feature_extraction.text import CountVectorizer



In [9]:
vectorizer = CountVectorizer(stop_words='english')

reviews_vector = vectorizer.fit_transform(data_reviews)
#vectorizer.get_feature_names()

In [10]:
reviews_vector.toarray().shape

(18999, 67017)

In [11]:

#vectorizer for the sentiment values

sentiment_scores = data['sentiment'].copy()

sentiment_vectorizer = CountVectorizer().fit_transform(sentiment_scores)
sentiment_vector=sentiment_vectorizer.toarray()

In [12]:
sentiment_vector_1D = np.zeros(len(sentiment_vector))


for i in range(len(sentiment_vector)):
    sentiment_vector_1D[i] = sentiment_vector[i][0]

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
reviews_train, reviews_test , sentiment_train, sentiment_test = train_test_split(
                                                                reviews_vector, sentiment_vector_1D,
                                                                train_size = 0.8 , random_state=42)

In [15]:
reviews_train.shape, reviews_test.shape, sentiment_train.shape, sentiment_test.shape

((15199, 67017), (3800, 67017), (15199,), (3800,))

$\textbf{Logistic Regression classifier}$

In [16]:

from sklearn.linear_model import LogisticRegression

LG = LogisticRegression()

LG.fit(reviews_train,sentiment_train)

sentiment_predict_score_LG=LG.predict(reviews_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [17]:
reviews_train[0].toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

In [18]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

In [19]:
sentiment_predict_score_LG

array([0., 1., 0., ..., 1., 1., 0.])

In [20]:
sentiment_test

array([0., 1., 0., ..., 1., 0., 0.])

from sklearn.metrics import precision_recall_fscore_support

y_true_LG = sentiment_test 
y_pred_LG = sentiment_predict_score_LG

LG_metrics=precision_recall_fscore_support(y_true_LG,y_pred_LG,average=None)
confusion_matrix(y_true_LG, y_pred_LG)

import seaborn as sn

plt.figure(figsize = (10,7))
sn.heatmap(confusion_matrix(y_true_LG, y_pred_LG),annot=True,cmap=plt.cm.Reds)
plt.title('confusion matrix without normalization Logistic Regression')

from sklearn.tree import DecisionTreeClassifier
DTC = DecisionTreeClassifier(random_state=0)

DTC_fit=DTC.fit(reviews_train,sentiment_train)

sentiment_predict_score_DTC=DTC_fit.predict(reviews_test)

y_true_DTC = sentiment_test 
y_pred_DTC = sentiment_predict_score_DTC

DTC_metrics=precision_recall_fscore_support(y_true_DTC,y_pred_DTC,average=None)
confusion_matrix(y_true_DTC, y_pred_DTC)

plt.figure(figsize = (10,7))
sn.heatmap(confusion_matrix(y_true_DTC, y_pred_DTC),annot=True,cmap=plt.cm.Greens)
plt.title('confusion matrix without normalization for Decision Tree')

$\textit{Random Forest Classifier}$

from sklearn.ensemble import RandomForestClassifier

RFC= RandomForestClassifier()
RFC_fit=RFC.fit(reviews_train,sentiment_train)
sentiment_predict_score_RFC=RFC_fit.predict(reviews_test)

y_true_RFC = sentiment_test 
y_pred_RFC = sentiment_predict_score_DTC

RF_metrics=precision_recall_fscore_support(y_true_RFC,y_pred_RFC,average=None)
confusion_matrix(y_true_RFC, y_pred_RFC)

from sklearn.pipeline import Pipeline

def different_classifiers(X_train,y_train,X_test,y_test):

    #------ LOGISTIC REGRESSION
    
    LG = LogisticRegression()

    LG.fit(X_train,y_train)

    sentiment_predict_score_LG=LG.predict(X_test)
    
    y_true_LG = y_test 
    y_pred_LG = sentiment_predict_score_LG

    LG_metrics=precision_recall_fscore_support(y_true_LG,y_pred_LG,average=None)
    LG_confusion_matrix=confusion_matrix(y_true_LG, y_pred_LG)
    
    #------ LOGISTIC REGRESSION
    
    
    
    #------ RANDOM FOREST CLASSIFIER
    
    RFC= RandomForestClassifier()
    RFC_fit=RFC.fit(X_train,y_train)
    sentiment_predict_score_RFC=RFC_fit.predict(X_test)

    y_true_RFC = y_test  
    y_pred_RFC = sentiment_predict_score_RFC

    LG_metrics=precision_recall_fscore_support(y_true_RFC,y_pred_RFC,average=None)
    RFC_confusion_matrix = confusion_matrix(y_true_RFC, y_pred_RFC)
    
    #------ RANDOM FOREST CLASSIFIER
    
    
    #------ DECISION TREE CLASSIFIER
    
    DTC = DecisionTreeClassifier(random_state=0 )  
    DTC_fit = DTC.fit(X_train,y_train)
    sentiment_predict_score_DTC=DTC_fit.predict(X_test)
                                 
    y_true_DTC = y_test 
    y_pred_DTC = sentiment_predict_score_DTC

    DTC_metrics=precision_recall_fscore_support(y_true_DTC,y_pred_DTC,average=None)
    DTC_confusion_matrix = confusion_matrix(y_true_DTC, y_pred_DTC)
    
    #------ DECISION TREE CLASSIFIER
    
    
    return LG_confusion_matrix,RFC_confusion_matrix,DTC_confusion_matrix

Logistic_Reg, Random_For,Decision_Tree = different_classifiers(reviews_train,
                                                              sentiment_train,
                                                              reviews_test,
                                                              sentiment_test)

print("Logistic Regression CM :\n",Logistic_Reg) , print("Random Forest CM:\n",Random_For), print("Decision Tree CM : \n", Decision_Tree)

In [33]:
reviews_train.shape, reviews_test.shape, sentiment_train.shape, sentiment_test.shape

((15199, 67017), (3800, 67017), (15199,), (3800,))

import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

keras_tokenizer = Tokenizer()

keras_tokenizer.fit_on_texts(data_reviews)

data_reviews_corpus = keras_tokenizer.word_index
# data_reviews_corpus_np = np.array(data_reviews_corpus)

keras_sequence = keras_tokenizer.texts_to_sequences(data_reviews)

len(keras_sequence[9])

padded_keras_sequence = pad_sequences(keras_sequence,padding='post')

In [39]:
#pad_sequences()

In [40]:
#keras_data_reviews=(keras_tokenizer.texts_to_matrix(data_reviews))
#

reviews_train_keras, reviews_test_keras , sentiment_train_keras, sentiment_test_keras = train_test_split(
                                                                padded_keras_sequence,
                                                                sentiment_vector_1D,
                                                                train_size = 0.8 ,
                                                                random_state=42)

DTC_keras = DecisionTreeClassifier(random_state=0)

DTC_fit_keras=DTC_keras.fit(reviews_train_keras,sentiment_train_keras)

sentiment_predict_score_DTC_keras=DTC_fit_keras.predict(reviews_test_keras)

y_true_DTC_keras = sentiment_test_keras 
y_pred_DTC_keras = sentiment_predict_score_DTC_keras

DTC_metrics_keras=precision_recall_fscore_support(y_true_DTC_keras,y_pred_DTC_keras,average=None)
confusion_matrix(y_true_DTC_keras, y_pred_DTC_keras)

Logistic_Reg_keras, Random_For_keras,Decision_Tree_keras = different_classifiers(
                                                              reviews_train_keras,
                                                              sentiment_train_keras,
                                                              reviews_test_keras,
                                                              sentiment_test_keras)


print("Logistic Regression CM_keras :\n",Logistic_Reg_keras) , print("Random Forest CM_keras:\n",Random_For_keras), print("Decision Tree CM_keras : \n", Decision_Tree_keras)

In [51]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize

sw = (stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [54]:
from gensim.parsing.preprocessing import remove_stopwords

text = "Nick likes to play football, however he is not too fond of tennis."
filtered_sentence = remove_stopwords(text)

#print(filtered_sentence)

In [55]:
for i in range (len(data_reviews)):
  data_reviews[i] = remove_stopwords(data_reviews[i])

In [56]:
data_reviews[0]

"reviewers mentioned watching oz episode you'll hooked right exactly happened thing struck oz brutality unflinching scenes violence set right word trust faint hearted timid pulls punches regards drugs sex violence hardcore classic use word called oz nickname given oswald maximum security state penitentary focuses mainly emerald city experimental section prison cells glass fronts face inwards privacy high agenda em city home aryans muslims gangstas latinos christians italians irish scuffles death stares dodgy dealings shady agreements far away main appeal fact goes shows wouldn't dare forget pretty pictures painted mainstream audiences forget charm forget romance oz doesn't mess episode saw struck nasty surreal couldn't ready watched developed taste oz got accustomed high levels graphic violence violence injustice crooked guards who'll sold nickel inmates who'll kill order away mannered middle class inmates turned prison bitches lack street skills prison experience watching oz comfortab

In [59]:
data['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa